In [73]:
import numpy as np
from tkinter import *
from tkinter import ttk

# Kemaru solver


La grille doit être rempli de façon à ce qu’une zone de 5 cases contienne les chiffres de 1 à 5, une zone de 4 cases contienne les chiffres de 1 à 4, etc. Un chiffre placé dans une case ne peut se trouver dans aucune des cases qui l’entourent (y compris en diagonale).

In [95]:
class Grille:
    def __init__(self, nb_rows, nb_cols):
        self.nb_rows = nb_rows
        self.nb_cols = nb_cols
        self.grid = np.zeros((nb_rows, nb_cols, 2), dtype=int)

    def set_value(self, row, col, subgrid_index, value = 0):
        if 0 <= row < self.nb_rows and 0 <= col < self.nb_cols:
            self.grid[row, col, 0] = value
            self.grid[row, col, 1] = subgrid_index
        else:
            raise IndexError("Row or column index out of bounds")

    def get_value(self, row, col):
        if 0 <= row < self.nb_rows and 0 <= col < self.nb_cols:
            return self.grid[row, col]
        else:
            raise IndexError("Row or column index out of bounds")
        
    def check_correctness(self):
        # Placeholder for correctness check logic
        nb_subgrids = np.max(self.grid[:, :, 1])
        assert nb_subgrids == 9, "Number of subgrids is not equal to 9"
        for subgrid_index in range(1, nb_subgrids + 1):
            subgrid = self.grid[self.grid[:, :, 1] == subgrid_index]
            print(f"Subgrid {subgrid_index} has {subgrid.shape[0]} elements")
            assert 0 < subgrid.shape[0] <= 5, f"Subgrid {subgrid_index} has invalid number of elements"
            for value in subgrid[:, 0]:
                assert 0 <= value <= subgrid.shape[0], f"Value {value} in subgrid {subgrid_index} is out of range"

    def possible_values(self, row, col):
        if self.grid[row, col, 0] != 0:
            return set()  # Cell already filled
        subgrid_index = self.grid[row, col, 1]
        subgrid = self.grid[self.grid[:, :, 1] == subgrid_index]
        values_in_subgrid = set(subgrid[:, 0])
        #print(f"Values in subgrid {subgrid_index}: {values_in_subgrid}")
        neighboring_values = set()
        for r in range(max(0, row - 1), min(self.nb_rows, row + 2)):
            for c in range(max(0, col - 1), min(self.nb_cols, col + 2)):
                if (r != row or c != col):
                    neighboring_values.add(self.grid[r, c, 0])
        #print(f"Neighboring values around ({row}, {col}): {neighboring_values}")
        possible_values = set(range(1, subgrid.shape[0] + 1)) - values_in_subgrid - neighboring_values
        #print(f"Possible values for ({row}, {col}): {possible_values}")
        return possible_values
    
    def subgrid_values(self, row, col):
        subgrid_index = self.grid[row, col, 1]
        # Get all positions in the same subgrid
        positions = np.argwhere(self.grid[:, :, 1] == subgrid_index)
        values = set()
        #print(f"Subgrid {subgrid_index} positions: {positions}")
        for pos in positions:
            r,c = pos
            if r != row or c != col:
                values = values.union(self.possible_values(r, c))

        only_values = self.possible_values(row, col)-values
        #print(f"Possible values for cell ({row}, {col}): {self.possible_values(row, col)}")
        #print(f"Values in other cells of subgrid {subgrid_index}: {values}")
        #print(f"Only values for cell ({row}, {col}): {only_values}")
        return only_values
        
        
    
    def solve(self):
        while True:
            progress_made = False
            for r in range(self.nb_rows):
                for c in range(self.nb_cols):
                    if self.grid[r, c, 0] == 0:
                        possible = self.possible_values(r, c)
                        if len(possible) == 1:
                            self.grid[r, c, 0] = possible.pop()
                            progress_made = True
                        only_values = self.subgrid_values(r, c)
                        if len(only_values) == 1:
                            self.grid[r, c, 0] = only_values.pop()
                            progress_made = True
            if not progress_made:
                break
    


    def display(self):
        root = Tk()
        frm = ttk.Frame(root, padding=10)
        frm.grid()
        color_list = ['red', 'orange', 'blue', 'wheat', 'white', 'yellow', 'green', 'pink', 'violet']
        for r in range(self.nb_rows):
            for c in range(self.nb_cols):
                value, subgrid_index = self.grid[r, c]
                cell_text = f"{value}" if value != 0 else ""
                label = ttk.Label(frm, text=cell_text, borderwidth=1, relief="solid", width=4, anchor="center", background=color_list[subgrid_index-1])
                label.grid(row=r, column=c, padx=1, pady=1)
        root.mainloop()



## Premier exemple

In [97]:
g = Grille(7, 6)
g.set_value(0, 0, 1, 2)
g.set_value(0, 1, 1)
g.set_value(0, 2, 2)
g.set_value(0, 3, 2)
g.set_value(0, 4, 3)
g.set_value(0, 5, 3)

g.set_value(1, 0, 1)
g.set_value(1, 1, 1)
g.set_value(1, 2, 2, 1)
g.set_value(1, 3, 2)
g.set_value(1, 4, 3, 3)
g.set_value(1, 5, 3)

g.set_value(2, 0, 1)
g.set_value(2, 1, 4)
g.set_value(2, 2, 4)
g.set_value(2, 3, 2)
g.set_value(2, 4, 5)
g.set_value(2, 5, 3)

g.set_value(3, 0, 4, 4)
g.set_value(3, 1, 4)
g.set_value(3, 2, 4, 3)
g.set_value(3, 3, 6)
g.set_value(3, 4, 5, 5)
g.set_value(3, 5, 5)

g.set_value(4, 0, 7)
g.set_value(4, 1, 8)
g.set_value(4, 2, 8)
g.set_value(4, 3, 6)
g.set_value(4, 4, 5)
g.set_value(4, 5, 5, 4)

g.set_value(5, 0, 7, 5)
g.set_value(5, 1, 7)
g.set_value(5, 2, 8, 5)
g.set_value(5, 3, 8)
g.set_value(5, 4, 9)
g.set_value(5, 5, 9)

g.set_value(6, 0, 7)
g.set_value(6, 1, 7, 4)
g.set_value(6, 2, 8)
g.set_value(6, 3, 9)
g.set_value(6, 4, 9, 2)
g.set_value(6, 5, 9)

g.display()


In [98]:
g.check_correctness()
g.solve()
g.display()

Subgrid 1 has 5 elements
Subgrid 2 has 5 elements
Subgrid 3 has 5 elements
Subgrid 4 has 5 elements
Subgrid 5 has 5 elements
Subgrid 6 has 2 elements
Subgrid 7 has 5 elements
Subgrid 8 has 5 elements
Subgrid 9 has 5 elements


## second exemple

In [101]:
g = Grille(7, 6)
g.set_value(0, 0, 1)
g.set_value(0, 1, 1)
g.set_value(0, 2, 2, 1)
g.set_value(0, 3, 3)
g.set_value(0, 4, 3)
g.set_value(0, 5, 3, 5)

g.set_value(1, 0, 1)
g.set_value(1, 1, 1)
g.set_value(1, 2, 2)
g.set_value(1, 3, 2)
g.set_value(1, 4, 3)
g.set_value(1, 5, 3)

g.set_value(2, 0, 4, 3)
g.set_value(2, 1, 1)
g.set_value(2, 2, 2, 5)
g.set_value(2, 3, 2)
g.set_value(2, 4, 5)
g.set_value(2, 5, 5, 5)

g.set_value(3, 0, 4)
g.set_value(3, 1, 6)
g.set_value(3, 2, 6)
g.set_value(3, 3, 6, 3)
g.set_value(3, 4, 5)
g.set_value(3, 5, 5)

g.set_value(4, 0, 4)
g.set_value(4, 1, 6)
g.set_value(4, 2, 6)
g.set_value(4, 3, 7)
g.set_value(4, 4, 7)
g.set_value(4, 5, 5)

g.set_value(5, 0, 4)
g.set_value(5, 1, 8, 1)
g.set_value(5, 2, 8)
g.set_value(5, 3, 7)
g.set_value(5, 4, 9)
g.set_value(5, 5, 9, 2)

g.set_value(6, 0, 4)
g.set_value(6, 1, 8)
g.set_value(6, 2, 8)
g.set_value(6, 3, 9)
g.set_value(6, 4, 9, 5)
g.set_value(6, 5, 9)
g.display()

In [102]:
g.check_correctness()
g.solve()
g.display()

Subgrid 1 has 5 elements
Subgrid 2 has 5 elements
Subgrid 3 has 5 elements
Subgrid 4 has 5 elements
Subgrid 5 has 5 elements
Subgrid 6 has 5 elements
Subgrid 7 has 3 elements
Subgrid 8 has 4 elements
Subgrid 9 has 5 elements
